In [1]:
import requests
from bs4 import BeautifulSoup
from dateutil import parser
import datetime
from urlparse import urljoin

In [2]:
def cli_green(fn):
    def wrapped(*args, **kwargs):
        return "\033[92m"+fn()
    return wrapped



def scrape_topic_page(topic_number):
    if not isinstance(topic_number, (basestring, int)):
        raise ValueError, "Topic number has to be int or basestring"
    soup = BeautifulSoup(
        requests.get(
            'http://netwars.pl/temat/{0!s}'.format(topic_number)
        ).text
    )
    return soup 
    


class NW(object):
    BASE_URL = 'http://netwars.pl/'
    OT_FORUM_NUMBER = '4'
    
    def __repr__(self):
        return '%s(%r)'%(self.__class__, self.username)
     
    def __init__(self, username, password):
        self.username = username
        self.password = password
        self.logged_in = False

    def login(self):
        payload = {
            'tnick':self.username,
            'tpass':self.password
        }
        
        nwsession = requests.session()
        nwsession.post(urljoin(self.BASE_URL,'login'), payload)
        self.nwsession = nwsession
        self.logged_in = True
    
    def logout(self):
        self.nwsession.post(urljoin(self.BASE_URL,'logout'))
        self.logged_in = False
        
    def topics(self):
        forum = self.nwsession.get(
            self.BASE_URL+'forum/'+self.OT_FORUM_NUMBER).text
        soup = BeautifulSoup(forum)
        return [topic.text for topic in soup.findAll('td',{'class':'topic'})]







        
        
        
        
        
    
    
    
        

        
# r = session.get('http://netwars.pl')
# nw_time = parser.parse(r.headers['date']).replace(tzinfo=None)

In [3]:
nw = NW('=SF=Vengeance','sorcery')

In [4]:
nw.login()

In [13]:
scrape_topic_page(170470).findAll('div')[50]

<div class="post" id="post_10"><div class="posthead"><div class="p2_data">2015-11-03 20:33:12</div><div class="p2_nick"><span class="numerek_posta">[#10]</span>\xa0<a class="nick" href="/user/18186">Ryceszzz</a></div></div><div class="post_body">9#<br/>\r\nokej, w zasadzie to nie wiem ;p <br/>\r\nja ju\u017c niestety nie posiadam</div></div>

In [5]:
forum = nw.topics()

/home/crimzoid/anaconda/lib/python2.7/site-packages/bs4/__init__.py:166: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

To get rid of this warning, change this:

 BeautifulSoup([your markup])

to this:

 BeautifulSoup([your markup], "lxml")

  markup_type=markup_type))


In [6]:
forum

[u'Ksi\u0105\u017cki #9',
 u'Poker #58',
 u'Filmy #53',
 u'Jokes #122',
 u'Muzyka i teledyski #2',
 u'Regulamin / FAQ / Lista moderator\xf3w',
 u'"Poradnik Jak Wkurwi\u0107 1000 Os\xf3b "',
 u'playstation polska mnie wychujalo[kkd][znwpz]',
 u'Imigranci #3',
 u'Wypiekacz do chleba ',
 u'PiS w\u0142adza "absolutna`15" #1',
 u'Dragon Ball Super',
 u'Wybaczanie',
 u'Przelewy do USA',
 u'Deska: zakup',
 u'M\xf3j brat zagin\u0105\u0142 [KKD]',
 u'polishsource.cz',
 u'Czujnik gazu i CO',
 u'impreza na schodach',
 u'Sprzedam / Kupi\u0119 #19',
 u'Si\u0142ownia - NW \u0107wiczy razem #138',
 u'Telegram',
 u'Jaki komputer, co polecacie? #6',
 u'PANIE PRZODEM',
 u'Co z nimi jest nie tak?',
 u'Dysk SSD - jak to dzia\u0142a?',
 u'nw best of 2015',
 u'Pi\u0142ka No\u017cna #39',
 u'Krak\xf3w (kkd)',
 u'Jaka to piosenka #245487',
 u'Programowanie, co wybra\u0107?',
 u'"To za Syri\u0119!" (zamkni\u0119ty)',
 u'NW POWER FACEBOOK',
 u'Mam c\xf3rk\u0119 : )',
 u'Dawna produkcja filmowa Dylar cup',
 u'NB